In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

# Simulation Parameters
L = 10.0   # Length of the domain
T = 1.0    # Total simulation time
Nx = 100   # Number of spatial grid points
Nt = 201   # Number of time steps
dx = L / (Nx - 1)  # Spatial step size
dt = T / Nt        # Time step size

# Monte Carlo Simulations
for o in range(1000):
    D = np.random.uniform(1, 100)  # Diffusion coefficient
    Kd = abs(np.random.uniform(D - 10, D + 10)) % 100  # Sorption coefficient
    R = 1 + Kd  # Retardation factor
    alpha = (D / R) * dt / dx**2  # Stability parameter

    print(f"Iteration {o}: D={D:.2f}, Kd={Kd:.2f}, R={R:.2f}, alpha={alpha:.4f}")

    if alpha > 0.5:
        print("Skipping: alpha > 0.5 (unstable)")
        continue  # Skip this iteration if unstable

    # Initialize concentration array
    C = np.zeros(Nx)  
    C[int(Nx / 4):int(Nx / 2)] = 1.0  # Initial condition: step function

    # Boundary conditions (Dirichlet: fixed concentration)
    C[0] = 0.0
    C[-1] = 0.0

    # Store concentration data over time
    C_all = np.zeros((Nt, Nx))  

    # Time-stepping loop
    for t in range(Nt):
        C_new = C.copy()
        for i in range(1, Nx - 1):
            C_new[i] = C[i] + alpha * (C[i + 1] - 2 * C[i] + C[i - 1])
        C = C_new

        # Apply boundary conditions at each time step
        C[0] = 0.0
        C[-1] = 0.0

        # Store current concentration profile
        C_all[t, :] = C

    # Save results to HDF5 file
    try:
        with h5py.File("diff_sorp.h5", "a") as f:
            group_name = f"{o}"
            while group_name in f:
                group_name += "_retry"
            g = f.create_group(group_name)
            g.create_dataset("D", data=D)
            g.create_dataset("Kd", data=Kd)
            g.create_dataset("R", data=R)
            g.create_dataset("data", data=C_all)
    except Exception as e:
        print(f"Error saving iteration {o}: {e}")

print("Simulation completed.")


Iteration 0: D=71.43, Kd=64.30, R=65.30, alpha=0.5333
Skipping: alpha > 0.5 (unstable)
Iteration 1: D=97.61, Kd=89.54, R=90.54, alpha=0.5257
Skipping: alpha > 0.5 (unstable)
Iteration 2: D=18.42, Kd=10.93, R=11.93, alpha=0.7530
Skipping: alpha > 0.5 (unstable)
Iteration 3: D=93.68, Kd=90.13, R=91.13, alpha=0.5012
Skipping: alpha > 0.5 (unstable)
Iteration 4: D=88.06, Kd=88.68, R=89.68, alpha=0.4788
Iteration 5: D=38.73, Kd=34.23, R=35.23, alpha=0.5360
Skipping: alpha > 0.5 (unstable)
Iteration 6: D=8.13, Kd=14.53, R=15.53, alpha=0.2554
Iteration 7: D=50.24, Kd=42.89, R=43.89, alpha=0.5582
Skipping: alpha > 0.5 (unstable)
Iteration 8: D=8.38, Kd=6.39, R=7.39, alpha=0.5528
Skipping: alpha > 0.5 (unstable)
Iteration 9: D=40.05, Kd=49.19, R=50.19, alpha=0.3892
Iteration 10: D=35.81, Kd=44.41, R=45.41, alpha=0.3845
Iteration 11: D=53.59, Kd=54.33, R=55.33, alpha=0.4723
Iteration 12: D=71.87, Kd=69.89, R=70.89, alpha=0.4943
Iteration 13: D=73.16, Kd=78.99, R=79.99, alpha=0.4460
Iteration 14:

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm
import h5py


def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)

visualize_burgers([i for i in range(100)], C_all, "a.gif")

100%|██████████| 201/201 [00:00<00:00, 5093.93it/s]


In [3]:
from scipy.stats import skewnorm

with h5py.File("diff_sorp.h5", "a") as f:
    for key in f.keys():
        u = f[key]["data"][:]
        noise = skewnorm.rvs(a=1, scale=0.2, size=u.shape)
        u_noise = u + noise
        f[key].create_dataset("noise", data=u_noise)